# Empaquetado de proyectos


¿Cómo distribuyo mi códgio?

- Uso en el interior de otros scripts/notebooks (con `import`)
- Uso como un script (tiene que instalarlo en el PATH, obviamente)
- Como un ejecutable autocontenido
- Uso como web API

## Módulos

Un módulo es un espacio de nombres (namespace):

- Funciones
- Constantes
- Clases
- Cualquier cosa a la que pueda ponerse un nombre en python


Tipicamente corresponde a un archivo único: something.py




### Estructura de `capitalize`

`capitalize` es un directorio con una serie de archivos de python que vamos a usar para ir construyendo un paquete paso a paso. Está compuesto de varios archivos:

- capital_mod.py: el archivo principal, que contiene un código que convierte las palabras de un archivo a mayúsculas. Contiene varias funciones: 

    - capitalize_line: transforma las componentes de una frase (string) a palabras en mayúsculas, salvo una serie de palabras funcionales.
    - load_special_words: lee un archivo con la lista de palabras que no transformará
    - capitalize: transforma un archivo de texto en otro con sus palabras en mayúscula pasando cada línea por capitalize_line
    - get_datafile_name: función auxiliar que devuelve la ruta completa del archivo donde se encuentran las palabras a evitar
    
- test_capital_mod.py: tests unitarios
- cap_script.py: la CLI y va a ser el ejecutable que se instale con el paquete
- cap_data.txt: las palabras auxiliares
- sample_text_file.txt: archivo de texto de prueba

### Ejercicio

Meterse en el directorio `capitalize` e importar el archivo capital_mod.py. Jugar con el espacio de nombres

In [ ]:
ls

In [ ]:
cd capitalize

In [ ]:
import capital_mod
capital_mod.capitalize_line('hola a todos')

### Ejercicio

- Desde la terminal, correr el script `cap_script.py` y comprobar que efectivamente funciona

Obviamente esta forma de importar código es poco práctica una vez que el código se vuelve algo más complejo... Y desde un punto de vista de mantenimiento y distribución es algo bastante incómodo.
Construyamos un paquete!

### Ejercicio

Duplicar la carpeta de `capitalize` y cambiarle el nombre a `capitalize_raw`. Esa será nuestra copia de seguridad de los archivos por si algo falla

## Paquetes

Un paquete es en esencia un módulo, excepto porque puede tener otros módulos (o paquetes) en su interior.

    a_package
       __init__.py
       module_a.py
       a_sub_package
         __init__.py
         module_b.py

Un paquete tipicamente corresponde a un subdirectorio con un archivo `__init__.py` y con un número indeterminado de archivos de python u otros directorios de paquetes.

> Ejercicio: salir de la carpeta capitalize e importar capitalize. Como no hay `__init__.py` pues no hace nada

In [ ]:
cd ..

In [ ]:
import capitalize

### `__init.py__`

Puede estar vacío o tener cualquier tipo de código en él. Ese código correrá al importarse el paquete.

Haciendo import paquete ejecutará el código en paquete/__init__.py, y los nombres definidos en él se acceden con paquete.nombre


### Ejercicio

Modificar el archivo `__init__.py` para que tenga algún nombre que poder usar o para que produzca un mensaje. Hay que borrar el compilado que ha creado, porque si no no lo modifica. Para ver los cambios, habrá que reiniciar el kernel.

In [ ]:
import capitalize



Los submódulos no los importa automáticamente: hay que importarlos explícitamente

### Ejercicio

importar el módulo capitalize.capitalize_mod.py y ver su espacio de nombres con `dir`

In [ ]:
import capitalize.capital_mod
dir(capitalize.capital_mod)

# El que s'ha escrit és un col·lectiu de tot el que té dins el directori capitalize.capital_mod, tot el què podem importar.

#és un equivalent a fer from capitalize import capital_mod

Una cuestión importante: es preferible importar los submódulos de forma _absoluta_ (`import capitalize.capital_mod`) que _relativa_ (`from capitalize import capitalize.mod`)
La razón es que de forma absoluta queda siempre claro de dónde proceden las funciones que utilicemos.

### El path de importaciones

El intérprete guarda una lista con todos los directorios donde busca paquetes



In [ ]:
import sys
for p in sys.path:
    print(p)

Todo módulo tiene un nombre `__file__` que apunta a dónde vive.

### Ejercicio

- Ver dónde está instalado capitalize haciendo `capitalize.__file__`
- Importar numpy y hacer lo mismo

In [ ]:
capitalize.__file__

In [ ]:
import numpy as np
np.__file__

## Organizando e instalando los paquetes

¿Qué tenemos que incluir en un paquete?

- Una colección de módulos
- Y su documentación
- Y sus tests
- Y los scripts de alto nivel 
- Los archivos de datos
- Y las instrucciones para construirlo e instalarlo


    package_name/
        bin/
        docs/ 
		CHANGES.txt 
		LICENSE.txt 
		MANIFEST.in 
		README.txt 
		setup.py 
		package_name/
              __init__.py
              module1.py
              module2.py
              test/
                  __init__.py
                  test_module1.py
                  test_module2.py

Como vemos, tenemos dos niveles. Primer nivel (package_name/):

- bin/: los scripts ejecutables (a veces se llama scripts/)
- docs/: la documentación
- CHANGES.txt: Qué hay de nuevo en esta versión
- LICENSE.txt: Cómo está licenciado (especificarlo!)
- MANIFEST.in: qué archivos que no sean código se necesitan
- README.txt: descripción de lo que hace el paquete. Usar markdown.
- `setup.py`: __el script que construye e instala el paquete__

Segundo nivel (package_name/package_name). El paquete en sí. Aquí es donde va el código
- test/: los tests unitarios para comprobar que todo funciona.
- `__init__.py` __lo que se ejecuta al importar el código__
- `module1.py`: los diferentes módulos del paquete

## El archivo `setup.py`

Es el archivo que describe el paquete y le dice a python cómo construirlo e instalarlo. 



In [ ]:
from setuptools import setup


setup(
        name='PackageName',
        version='0.1.0',
        author='An Awesome Coder',
        author_email='aac@example.com',
        packages=['package_name', 'package_name.test'],
        scripts=['bin/script1','bin/script2'],
        url='http://pypi.python.org/pypi/PackageName/',
        license='LICENSE.txt',
        description='An awesome package that does something',
        long_description=open('README.txt').read(),
        install_requires=[
        "Django >= 1.1.1",
        "pytest", ],
)

### ¿Qué es setuptools?

Es el módulo de la base de python que permite instalar paquetes. Antes de usaba el distutils: `from distutils.core import setup`

Pero es cada vez menos usado, y ha sido superado por setuptools: `from setuptools import setup`

Los campos de la función `setup` son bastante explicativos:

- name: nombre del paquete
- version: string que nos dice la versión del paquete. Es importante puesto que el paquete instalado se actualizará cuando la versión cambie
- author: autor del código
- packages: los paquetes contenidos en el módulo. Será lo que podrá ser importado
- scripts: los scripts contenidos en el código. Serán instalados en el directorio apropiado para que puedan ser ejecutados desde la línea de comandos
- url: la url del paquete (de github o Pypi)
- license: el archivo donde se especifica la licencia
- description: para qué sirve el paquete, en pocas palabras
- long_description: descripción larga de las funcionalidades del paquete. Generalmente es el archivo `README.txt`
- install_requires: una lista con las dependencias mínimas que necesita el paquete para funcionar. Es una de los campos más críticos
- keywords: infromación adicional sobre el pauete
- python_requires: especifica la versión concreta de python


## Ejecutando el setup.py

Hay dos formas de hacerlo. La forma tradicional es ejecutarlo directamente:

    python setup.py build
    python setup.py install

Pero es mucho mejor instalarlos a través de pip, ya que gestionará de un modo mucho mejor las dependencias:

    pip install .
   
Que instala el módulo que se encuentre en el directorio actual. La sintaxis general especifica la ruta concreta del archivo:

    pip install /path/to-source/tree  

### Instalación en modo desarrollo

Algo muy útil: instala el paquete, pero todos los cambios que se hagan tienen efecto inmediato.

    python setup.py develop
    
    pip install -e .



### install_requires


Este es la campo que le dice a `pip` qué dependecias tiene nuestro paquete. 

> `pip` no utiliza el archivo `requirements.txt` (o `environment.yml` en `conda`) para conocer las dependencias del paquete, sino que busca esta información en `setup.py`

En principio debería listar solo las dependencias directas. Recordemos que en `requirements.txt` era necesario incluir también las indirectas.


### `requirements.txt` vs. `setup.py`


¿¿Son necesarios ambos??

En realidad, depende para qué lo estemos usando. A grandes rasgos:

- Si el paquete es solo para desarrollo y no estamos pensando en distribuirlo, `requirements.txt` o `environment.yml` es suficiente
- Si se piensa distribuir el paquete pero se desarrolla solo en una máquina, `setup.py/setup.cfg` es suficiente
- Si se está desarrollando en varias máquinas y se piensa distribuir, necesitará ambos archivos

> Resumiendo: 
> - Para desarrollo:  `requirements.txt` o `environment.yml`
> - Para distribución: `setup.py/setup.cfg` 

### setup.cfg

En ocasiones, para escribir un código más limpio y comprensible, en lugar de estructurar el archivo `setup.py` como hemos visto se usa el archivo `setup.cfg` Es sencillamente un archivo. `ini` que contiene los valores por defecto para `setup.py`. Lo suyo es especificar las opciones del `setup.py` en el `setup.cfg` y utilizar el primero solo como el CLI.



Ejemplo de setup.cfg:

    # setup.cfg file at the root directory
    [metadata]
    name = examplepackage
    version = 1.0.1
    author = Giorgos Myrianthous
    description = This is an example project
    long_description = This is a longer description for the project
    url = https://medium.com/@gmyrianthous
    keywords = sample, example, setuptools
    [options]
    python_requires = >=3.7, <4
    install_requires = 
        pandas
    [options.extras_require]
    test = 
        pytest
        coverage
    [options.package_data]
    sample = 
        example_data.csv'

### setup.py con setup.cfg

Si todas las opciones están especificadas en el `setup.cfg`, se puede hacer un `setup.py` dummy que solamente llame al método `setup()`:

    from setuptools import setup
    if __name__ == '__main__':
        setup()

## Creando un paquete

Todo lo que no sea un script simple merece ser estructurado como paquete:

- Crear la estructura de directorios
- Escribir el setup.py
- `pip install -e .`
- Hacer los tests

Convirtamos capitalize en un paquete

Primero, estructurémoslo.

    capitalize/
        setup.py 
        bin/
            cap_script.py
        docs/ 

        capitalize/
            __init__.py
            capital_mod.py
            test/
                 __init__.py
                 test_capital_mod.py
            data/
                cap_data.txt
                sample_text_file.txt

A continuanción, escribamos el `setup.py`

    from setuptools import setup
    setup(name='capitalize',
          version='1.0',
          # list folders, not files
          packages=['capitalize',
                    'capitalize.test'],
          scripts=['bin/cap_script.py'],
          package_data={'capitalize': ['data/*']},
          )
              
E instalémoslo!

## Veamos si esto funciona

Lo que vamos a hacer a continuación es ejecutar comandos desde la teminal para instalar este paquete

In [ ]:
ls

In [ ]:
cd capitalize

In [ ]:
pip install -e .

### Lo primero de todo: ¿dónde ha metido esto?

Ojo: resetear el kernel para que aparezca!


In [ ]:
import capitalize
print(capitalize.__file__)

In [ ]:
conda list

Pero si intentamos ejecutarlo desde la terminal con `cap_script.py` nos va a dar problemas porque no encuentra las cosas. Arreglémoslo.

En cap_script.py:

    import capital_mod

debería ser:

    from capitalize import capital_mod

Y lo mismo en test_capital_mod.py


Ahora tampoco encuentra el archivo de datos en `capital_mod.py` ya que lo hemos cambiado de localización. Arreglémoslo. Hay que cambiar la línea 32:

    return Path(__file__).parent / "cap_data.txt"

por:

    return Path(__file__).parent / "data/cap_data.txt"
    
Y ahora ya debería funcionar todo. (resetear el kernel antes de importar de nuevo desde jupyter)

In [1]:
import capitalize

Módulo importado


In [ ]:
import capitalize.capital_mod

In [ ]:
import capitalize.test.test_capital_mod

In [ ]:
capitalize.capital_mod.capitalize_line('hola amigos')

In [ ]:
from capitalize import capital_mod

In [ ]:
capital_mod.capitalize_line('hola a todos')

Y los ejecutables están en el path

In [ ]:
import os
os.system('cap_script.py')

Los archivos de `/data` pueden verse usando el módulo `pkgutil`

In [ ]:
import pkgutil

In [ ]:
pkgutil.get_data( 'capitalize','data/sample_text_file.txt')

Para desinstalarlo, no hay más que correr 

In [ ]:
pip uninstall capitalize

El ejecutable no se borrar automáticamente; hay que hacerlo manualmente. Está en 'opt/anaconda/bin'

## Instalando desde Git

Una forma de distribuir nuestro código es instalarlo desde Git si tenemos subido nuestros archivos en un repositorio remoto. Para ello tenemos que tener instalados `git` y `pip` (lo cual es más que probable) y usar el siguiente comando:


In [ ]:
pip install git+https://github.com/thebluenote/capitalize.git

> Ojo con la sintaxis de la URL

### Ejercicio

- Desinstalar capitalize
- Subirlo a github
- Instalarlo desde allí
- Comprobar que todo funciona

In [ ]:
import capitalize

In [ ]:
conda list

In [ ]:
import capitalize.test
dir(capitalize.test)

In [ ]:
import capitalize.test.test_capital_mod

In [ ]:
capitalize.test.test_capital_mod.test_capitalize

In [ ]:
capitalize.test.test_capital_mod

## Instalando desde pypi

Cuando nuestro módulo esté ya para publicar, podemos subirlo a `pypi` para que sea instalable con `pip`. Hay que registrarse en https://pypi.org/account/register/

Tiene la enorme ventaja que se encarga de instalar las dependencias directas (e indirectas) necesarias para el funcionamiento del paquete. Bueno, siempre que las hayamos detallado en el `setup.py`

Para ello necesitamos la librería `twine`

La mejor manera es hacer un Makefile como el siguiente:

    clean:
        rm -rf dist/
    dist: clean
        python setup.py sdist bdist_wheel
    release: dist
        twine upload dist/*
 
 $ make release

- `clean` elimina las posibles versiones de distribución previas
- `dist` crea la nueva distribución a partir del archivo `setup.py``
- `release` sube la distribución al repositorio a través de la librería `twine`

### Ejercicio

- Desinstalar capitalize
- Subirlo a `pypi`
- Instalarlo desde allí
- Comprobar que todo funciona